#### Worked Example: Counting Organization in a Database

In [8]:
#!/usr/bin/env python

# Counting Organizations

# This application will read the mailbox data (mbox.txt) count up the number
# email messages per organization (i.e. domain name of the email address) using
# a database with the following schema to maintain the counts.
'''
CREATE TABLE Counts (org TEXT, count INTEGER)
'''
# You can use this code as a starting point for your application:
# http://www.pythonlearn.com/code/emaildb.py.

# The data file for this application is the same as in previous assignments:
# http://www.pythonlearn.com/code/mbox.txt.

# Because the sample code is using an UPDATE statement and committing the
# results to the database as each record is read in the loop, it might take as
# long as a few minutes to process all the data. The commit insists on
# completely writing all the data to disk every time it is called.

# The program can be speeded up greatly by moving the commit operation outside
# of the loop. In any database program, there is a balance between the number of
# operations you execute between commits and the importance of not losing the
# results of operations that have not yet been committed.

import sqlite3

# create connection object
conn = sqlite3.connect('org_count.sqlite')

# create cursor object
cur = conn.cursor()

# delete table if it exists
cur.execute('''
DROP TABLE IF EXISTS Counts''')

# create table with attributes org and count
cur.execute('CREATE TABLE Counts (org TEXT, count INTEGER)')

# read file name from cmdline
fname = input('Enter file name: ')

# assign default if none entered in cmdline
if ( len(fname) < 1 ) : fname = 'mbox.txt'
    
# create handle to open filename entered
fh = open(fname)

# loop through to filter for email, then filter for org
for line in fh:
    if not line.startswith('From: ') : continue
    org = line.split()[1].split('@')[1]
    # retrieve existing data
    cur.execute('SELECT count FROM Counts WHERE org = ? ', (org, ))
    
    # query database
    row = cur.fetchone()
    
    # if it don't exists, insert else update  
    if row is None:
        cur.execute('INSERT INTO Counts (org, count) VALUES ( ?, 1 )', ( org, ))
    else :
        cur.execute('UPDATE Counts SET count=count+1 WHERE org = ?', (org, ))

    # This statement commits outstanding changes to disk each
    # time through the loop - the program can be made faster
    # by moving the commit so it runs only after the loop completes
    conn.commit()

# show count  
# https://www.sqlite.org/lang_select.html
sqlstr = 'SELECT org, count FROM Counts ORDER BY count DESC LIMIT 10'

print("Counts:")
for row in cur.execute(sqlstr) :
    print(str(row[0]), row[1])

# close cursor
cur.close()

Enter file name:  mbox.txt


Counts:
iupui.edu 536
umich.edu 491
indiana.edu 178
caret.cam.ac.uk 157
vt.edu 110
uct.ac.za 96
media.berkeley.edu 56
ufp.pt 28
gmail.com 25
et.gatech.edu 17
